In [1]:
import yaml
def load_credentials(credentials_filename: str):
    with open(f'{credentials_filename}.yaml','r') as f:
        credentials = yaml.safe_load(f)
        print(credentials)
        return credentials
# load_credentials('credentials')
db_credentials = load_credentials('credentials')
print(db_credentials)
    

{'RDS_HOST': 'eda-projects.cq2e8zno855e.eu-west-1.rds.amazonaws.com', 'RDS_PASSWORD': 'EDAonlinecustomer', 'RDS_USER': 'onlinecustomeranalyst', 'RDS_DATABASE': 'web_data', 'RDS_PORT': 5432}
{'RDS_HOST': 'eda-projects.cq2e8zno855e.eu-west-1.rds.amazonaws.com', 'RDS_PASSWORD': 'EDAonlinecustomer', 'RDS_USER': 'onlinecustomeranalyst', 'RDS_DATABASE': 'web_data', 'RDS_PORT': 5432}


In [2]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

class RDSDatabaseConnector:
    def __init__(self, credentials: dict):
        self.host = credentials.get('RDS_HOST')
        self.user = credentials.get('RDS_USER')
        self.password = credentials.get('RDS_PASSWORD')
        self.dbname = credentials.get('RDS_DATABASE')
        self.port = credentials.get('RDS_PORT')
    
    # Validate and convert the port value
        if self.port is None:
            raise ValueError("Port must be provided.")
        
        try:
            self.port = int(self.port)
        except ValueError:
            raise ValueError("Port must be an integer.")
        
        self.engine = None

    def initialize_sqlalchemy_engine(self):
        connection_string = f"postgresql://{self.user}:{self.password}@{self.host}:{self.port}/{self.dbname}"
        self.engine = create_engine(connection_string)
        return self.engine

    def fetch_data(self, query: str) -> pd.DataFrame:
        if self.engine is None:
            raise Exception("Engine not initialized. Call initialize_sqlalchemy_engine() first.")
        return pd.read_sql_query(query, self.engine)

    def get_customer_activity(self) -> pd.DataFrame:
        if self.engine is None:
            raise Exception("Engine not initialized. Call initialize_sqlalchemy_engine() first.")
        query = "SELECT * FROM customer_activity"
        return pd.read_sql_query(query, self.engine)
    
connector = RDSDatabaseConnector(db_credentials)
connector.initialize_sqlalchemy_engine()
customer_activity = connector.get_customer_activity()
print(customer_activity)
    

       administrative  administrative_duration  informational  \
0                 0.0                 0.000000              0   
1                 0.0                 0.000000              0   
2                 2.0                99.400000              0   
3                 0.0                 0.000000              0   
4                 0.0                 0.000000              0   
...               ...                      ...            ...   
12325             NaN               134.000000              4   
12326             5.0                      NaN              1   
12327             0.0                 0.000000              0   
12328             5.0                98.000000              0   
12329             8.0               244.160414              2   

       informational_duration  product_related  product_related_duration  \
0                        0.00              4.0                       0.0   
1                        0.00             26.0                     

In [3]:
customer_activity

,administrative,administrative_duration,informational,informational_duration,product_related,product_related_duration,bounce_rates,exit_rates,page_values,month,operating_systems,browser,region,traffic_type,visitor_type,weekend,revenue
0,0.0,0.000000,0,0.00,4.0,0.0,0.2000,0.200,0.0,May,Android,Google Chrome,North America,Twitter,Returning_Visitor,False,False
1,0.0,0.000000,0,0.00,26.0,876.0,0.0000,0.026,0.0,Nov,Windows,Google Chrome,Western Europe,Google search,Returning_Visitor,True,False
2,2.0,99.400000,0,0.00,19.0,368.0,0.0396,0.052,0.0,Sep,Windows,Google Chrome,Asia,Twitter,Returning_Visitor,False,False
3,0.0,0.000000,0,0.00,20.0,1432.0,0.0248,0.028,1.8,Dec,Windows,Google Chrome,Western Europe,Instagram ads,Returning_Visitor,False,True
4,0.0,0.000000,0,0.00,33.0,694.0,0.0141,0.032,0.0,May,Android,Google Chrome,North America,Instagram ads,Returning_Visitor,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,NaN,134.000000,4,112.22,48.0,1898.0,0.0030,0.007,0.0,Nov,Windows,Google Chrome,Western Europe,Google search,Returning_Visitor,True,False
12326,5.0,NaN,1,0.00,67.0,2202.0,0.0276,0.069,0.0,Aug,Windows,Google Chrome,Southern Africa,Facebook ads,Returning_Visitor,False,False
12327,0.0,0.000000,0,0.00,13.0,316.0,0.0000,0.017,0.0,Dec,Windows,Google Chrome,South America,Google search,New_Visitor,False,False
12328,5.0,98.000000,0,0.00,46.0,1469.0,0.0442,0.069,0.0,Sep,Windows,Google Chrome,North America,Facebook ads,Returning_Visitor,False,False


In [4]:
customer_activity.to_csv('customer.csv', index=False)

In [ ]:
import pandas as pd
   
def load_data_from_csv(customer):
        try:
            df = pd.read_csv('customer.csv')
            print(f"Data successfully loaded from {'customer.csv'}")
            return df
        except Exception as e:
            print(f"Error loading data from {'customer.csv'}: {e}")
            return None
df = load_data_from_csv('customer.csv')
print(df.head())  

Data successfully loaded from customer.csv
   administrative  administrative_duration  informational  \
0             0.0                      0.0              0   
1             0.0                      0.0              0   
2             2.0                     99.4              0   
3             0.0                      0.0              0   
4             0.0                      0.0              0   

   informational_duration  product_related  product_related_duration  \
0                     0.0              4.0                       0.0   
1                     0.0             26.0                     876.0   
2                     0.0             19.0                     368.0   
3                     0.0             20.0                    1432.0   
4                     0.0             33.0                     694.0   

   bounce_rates  exit_rates  page_values month operating_systems  \
0        0.2000       0.200          0.0   May           Android   
1        0.0000      